In [1]:
from azureml.core import Workspace, Run
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.0.1005480


In [2]:
ws = Workspace.get_or_create(name='mladsdemo', location='eastus2', subscription_id='6a275e9b-a3e6-42c2-b158-760d27f5cb83', resource_group='mladsdemo')
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: mladsdemo
Azure region: eastus2
Subscription id: 6a275e9b-a3e6-42c2-b158-760d27f5cb83
Resource group: mladsdemo


In [3]:
# get the text data from the github repo and unzip it
from fit_and_store_pipeline import unzip_file_here
import urllib
import os

if not os.path.isfile('./attack_data.csv'):
    if not os.path.isfile('./text_data.zip'): 
        urllib.request.urlretrieve('https://activelearning.blob.core.windows.net/activelearningdemo/text_data.zip', 'text_data.zip')
    unzip_file_here('text_data.zip')

if not os.path.isfile('miniglove_6B_50d_w2v.txt'):
    unzip_file_here('miniglove_6B_50d_w2v.zip')
    
print('Data files here')

C:\Users\tosingli\AppData\Local\conda\conda\envs\myenv\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Data files here


In [5]:
from fit_and_store_pipeline import *
global reloaded_model
script_main()

[[842  15]
 [ 96  47]]


NameError: name 'reloaded_model' is not defined

In [15]:
%%writefile myenv.yml
name: myenv
channels:
  - defaults
dependencies:
  - pip:
    - numpy
    - scikit-learn
    # Required packages for AzureML execution, history, and data preparation.
    - --extra-index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1
    - azureml-core
    - gensim

Writing myenv.yml


In [14]:
# you may need to register the provider in your subscription 
# az provider register -n Microsoft.ContainerInstance

from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 7, 
                                               tags = ['MLADS 2018 Text Workshop'], 
                                               description = 'Sample container deployment')

In [ ]:
%%time
# this will take 5-10 minutes to finish
service = ws.deploy_webservice(name = 'aci-attack-clsfr',
                               deploy_config = aciconfig,
                               target = None,
                               models = ['rf_attack_classifier_pipeline.pkl'],
                               runtime = 'python',
                               conda_file = 'myenv.yml',
                               driver = 'fit_and_store_pipeline.py')
service.wait_for_deployment(show_output = True)

In [ ]:
# verify the deployment
print('web service is hosted in ACI:', service.scoring_uri)

In [ ]:
# you can also use "az container list" command to find the ACI being deployed
!az container list

In [ ]:
# Now score the webservice

import json

test_attacks = [ 'You are scum.', 'I like your shoes.', 'You are pxzx.', 
                 'Your mother was a hamster and your father smelt of elderberries',
                 'One bag of hagfish slime, please']

# score the first row from the test set.
test_samples = json.dumps({"data": test_attacks})
test_samples = bytes(test_samples, encoding = 'utf8')

service.run(input_data = test_samples)